<a href="https://colab.research.google.com/github/somyaranjan26/DeepLearning/blob/master/VGG19_3_Classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Connect to Google Drive to access Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Path of dataset directory

In [ ]:
import os
os.chdir('/content/drive/My Drive/Deep Learning Internship/Training')

### Import all dependencies

In [ ]:
from __future__ import print_function, division
from builtins import range, input

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from glob import glob
import pandas as pd

### Define Few Parameters

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

DATADIR = ""
IMG_SIZE = 224
CATEGORIES = ["covid", "Normal","Pneumonia"]

### Fetch Images and Class Labels from Files (This might take a while)

In [ ]:
training_data = []

def create_training_data():
    for category in CATEGORIES: 

        path = os.path.join(DATADIR,category)                            # create path to covid19, normal and pneomonia
        class_num = CATEGORIES.index(category)                           # get the classification  (0,1 or 2). 0=covid 1=normal and 2=pneomonia

        for img in tqdm(os.listdir(path)):                               # iterate over each image per covid19, normal and pneomonia
            try:
                img_array = cv2.imread(os.path.join(path,img) )          # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])             # add this to our training_data
            except Exception as e:                                       # in the interest in keeping the output clean...
                pass
            #except OSError as e:
            #    print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #    print("general exception", e, os.path.join(path,img))

create_training_data()

print(len(training_data))

In [ ]:
import random

random.shuffle(training_data)

In [ ]:
x = []
y = []

for features,label in training_data:
    x.append(features)
    y.append(label)

#print(x[1].reshape(-1,224,224,3))
#print(x[1].reshape(-1, IMG_SIZE, IMG_SIZE, 3))

x = np.array(x)
y=np.array(y)

In [ ]:
import keras
from keras import layers

### Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2, random_state = 0)
print(yTrain.shape, yTest.shape)

yTrain = keras.utils.to_categorical(yTrain,3)
yTest = keras.utils.to_categorical(yTest, 3)

### **Normalization**
#### Model takes images in the form of array of pixels. Hence convert into array and *normalize*

In [ ]:
# Convert the images to float and scale it to a range of 0 to 1
xTrain = xTrain.astype('float32')
xTest = xTest.astype('float32')
xTrain /= 255.
xTest /= 255.

### **Building and Visualizing model**

In [ ]:
vggModel = VGG19(weights="imagenet", include_top=False, input_shape=(224,224,3))

outputs = vggModel.output
outputs = Flatten(name="flatten")(outputs)
outputs = Dropout(0.2)(outputs)
outputs = Dense(3, activation="softmax")(outputs)

model = Model(inputs=vggModel.input, outputs=outputs)

for layer in vggModel.layers:
    layer.trainable = False

model.compile(
        loss='categorical_crossentropy', 
        optimizer='adam', 
        metrics=['accuracy']
)

In [ ]:
model.summary()

### **Image Augmentation**
To train on images at different positions, angles, flips, etc

In [ ]:
train_aug = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

### **Training the model**

In [ ]:
history = model.fit(train_aug.flow(xTrain, yTrain, batch_size=32),
                    validation_data=(xTest, yTest),
                    validation_steps=len(xTest) / 32,
                    steps_per_epoch=len(xTrain) / 32,
                    epochs=200)

In [ ]:
model.save('vgg_3_classes.h5')

In [ ]:
model.save_weights('vggweight_3_classes.hdf5')

In [ ]:
model = load_model('vgg_3_classes.h5')

### **Making Predicions**

In [ ]:
y_pred = model.predict(xTest, batch_size=32)

### Convert to Binary classes

In [ ]:
test_labels = np.argmax(yTest, axis=1)
test_pred = np.argmax(y_pred, axis=1)

In [ ]:
test_acc = sum(test_labels == test_pred) / len(test_labels)
print('Accuracy: %.3f' % test_acc)